In [2]:
import datetime
from dateutil.relativedelta import relativedelta


# Custom exception for balance-related errors
class BalanceException(Exception):
    pass


# Represents a generic bank account
class BankAccount:
    def __init__(self, initialAmount, acctName):
        self.balance = initialAmount
        self.name = acctName
        self.planned_transactions = []
        self.transaction_history = []  # New attribute for transaction history
        print(
            f"\nAccount '{self.name}' created.\nBalance = ${self.balance:.2f}")

    # Get the current balance of the account
    def getBalance(self):
        print(f"\nAccount '{self.name}' balance = {self.balance:.2f}")

    # Deposit a specified amount into the account
    def deposit(self, amount):
        self.balance = self.balance + amount
        print("\nDeposit complete.")
        self.getBalance()

    # Check if a transaction of a specified amount is viable
    def viableTransaction(self, amount):
        if self.balance >= amount:
            return
        else:
            raise BalanceException(
                f"\nSorry, account '{self.name}' only has a balance of ${self.balance:.2f}")

    # Withdraw a specified amount from the account
    def withdraw(self, amount):
        try:
            self.viableTransaction(amount)
            self.balance = self.balance - amount
            print("\nWithdraw complete.")
            self.getBalance()
        except BalanceException as error:
            print(f"\nWithdraw interrupted: {error}")

    # Transfer a specified amount from this account to another account
    def transfer(self, amount, account):
        try:
            print("\n**********\n\nBeginning Transfer...")
            self.viableTransaction(amount)
            self.withdraw(amount)
            account.deposit(amount)
            self.record_transaction(f"Transfer to {account.name}", amount)
            print("\nTransfer Complete!\n\n**********")

        except BalanceException as error:
            print(f"Transfer interrupted. {error}")

    # Record a transaction in the transaction history
    def record_transaction(self, description, amount):
        transaction = {
            'date': datetime.datetime.now(),
            'description': description,
            'amount': amount
        }
        self.transaction_history.append(transaction)

    # Generate a report of transaction history
    def generate_transaction_report(self):
        print(f"\nTransaction Report for Account '{self.name}':")
        for transaction in self.transaction_history:
            print(
                f"{transaction['date']} - {transaction['description']}: ${transaction['amount']:.2f}")

    # Plan a future transaction with a specified amount, target account, and planned date
    def plan_transaction(self, amount, target_account, planned_date_str):
        planned_date = PlannedTransaction.parse_date(planned_date_str)
        planned_transaction = PlannedTransaction(
            amount, target_account, planned_date)
        self.planned_transactions.append(planned_transaction)
        print(
            f"\nTransaction planned for {planned_date_str}. Amount: ${amount:.2f}")

    # Execute planned transactions that are due on the current date
    def execute_planned_transactions(self, current_date):
        transactions_to_execute = []
        for planned_transaction in self.planned_transactions:
            if planned_transaction.planned_date <= current_date:
                transactions_to_execute.append(planned_transaction)

        for planned_transaction in transactions_to_execute:
            self.transfer(planned_transaction.amount,
                          planned_transaction.target_account)
            self.planned_transactions.remove(planned_transaction)

        if transactions_to_execute:
            print("\nPlanned transactions executed.")
        else:
            print("\nNo planned transactions for today.")

    # Automated Transfers: Implement a function for automated transfers based on a plan
    def automated_transfers(self, amount, target_account, frequency, start_date, end_date):
        current_date = datetime.datetime.now()
        planned_date = PlannedTransaction.parse_date(start_date)
        end_date = PlannedTransaction.parse_date(end_date)

        while planned_date <= end_date:
            if planned_date >= current_date:
                self.plan_transaction(amount, target_account, planned_date.strftime("%Y-%m-%d"))
            if frequency == 'monthly':
                planned_date = planned_date + relativedelta(months=1)
            elif frequency == 'weekly':
                planned_date = planned_date + datetime.timedelta(weeks=1)

    # Notifications: Add a function for notifying the user about important events
    def notify_user(self, message):
        print(f"\nNotification for Account '{self.name}': {message}")


# Represents a planned financial transaction
class PlannedTransaction:
    def __init__(self, amount, target_account, planned_date):
        self.amount = amount
        self.target_account = target_account
        self.planned_date = planned_date

    # Static method to parse a date string into a datetime object
    @staticmethod
    def parse_date(date_str):
        return datetime.datetime.strptime(date_str, "%Y-%m-%d")


# Example usage:
if __name__ == "__main__":
    # Create accounts
    account1 = BankAccount(1000, "Bob Wilson")
    account2 = BankAccount(2000, "Nancy Shields")

    # Plan transactions
    account1.plan_transaction(500, account2, "2023-01-05")
    account2.plan_transaction(300, account1, "2023-01-10")

    # Simulate passage of time (e.g., to the current date)
    current_date = datetime.datetime.now()

    # Execute planned transactions for both accounts
    account1.execute_planned_transactions(current_date)
    account2.execute_planned_transactions(current_date)

    # Generate transaction reports for both accounts
    account1.generate_transaction_report()
    account2.generate_transaction_report()



Account 'Bob Wilson' created.
Balance = $1000.00

Account 'Nancy Shields' created.
Balance = $2000.00

Transaction planned for 2023-01-05. Amount: $500.00

Transaction planned for 2023-01-10. Amount: $300.00

**********

Beginning Transfer...

Withdraw complete.

Account 'Bob Wilson' balance = 500.00

Deposit complete.

Account 'Nancy Shields' balance = 2500.00

Transfer Complete!

**********

Planned transactions executed.

**********

Beginning Transfer...

Withdraw complete.

Account 'Nancy Shields' balance = 2200.00

Deposit complete.

Account 'Bob Wilson' balance = 800.00

Transfer Complete!

**********

Planned transactions executed.

Transaction Report for Account 'Bob Wilson':
2024-01-03 11:07:09.562935 - Transfer to Nancy Shields: $500.00

Transaction Report for Account 'Nancy Shields':
2024-01-03 11:07:09.562935 - Transfer to Bob Wilson: $300.00
